In [1]:
#Libraries
from typing import Optional, Any
import warnings
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tifffile import TiffFile, TiffWriter
import numpy as np
import pandas as pd
import os
from google.colab import drive
from sklearn  import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestCentroid
from joblib import dump, load
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
# Import the function for reading an image
from skimage.io import imread
from skimage.color import rgb2yiq
from skimage.color import rgb2gray
from sklearn.preprocessing import label_binarize

import h5py

import cv2
from skimage.exposure import equalize_hist
from matplotlib import pyplot
from matplotlib import colors as cls

import numpy as np

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import roc_curve, auc
from sklearn.cluster import KMeans

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import os 

#LIBRARIS USED IN VIEWs FILE DJango
from PIL import Image
from io import BytesIO
from django.core.files import File
import os
from django.conf import settings
from django.shortcuts import render
from django.http import HttpResponse
from django.shortcuts import render
from django.views.generic import TemplateView
from django.views import View
from django.http import JsonResponse
import json
from typing import Optional, Any
import warnings
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tifffile import TiffFile, TiffWriter
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestCentroid
from joblib import dump, load
from skimage.measure import label, regionprops
# Import the function for reading an image
from skimage.io import imread
from skimage.color import rgb2yiq
from skimage.color import rgb2gray


# import cv2
from skimage.exposure import equalize_hist
from matplotlib import pyplot
from matplotlib import colors as cls

import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from .models import *

from . import corefiles
import h5py
from classifierapp.dataset.extractdataset import loadDataset
from .corefiles import read_mtiff,read_stiff
from tifffile import TiffFile, TiffWriter
from classifierapp.models_fold.getmodel import loadModel


Load MODEL

In [ ]:

def loadModel():
    __location__ = os.path.realpath(
    os.path.join(os.getcwd(), os.path.dirname(__file__)))

    return  load(os.path.join(__location__,'kbest35_mlp10000logisticmodel.joblib')) , load(os.path.join(__location__,'kbest35_n10000slectkb.joblib'))

# LOAD DATASET

In [ ]:

def loadDataset():
    __location__ = os.path.realpath(
    os.path.join(os.getcwd(), os.path.dirname(__file__)))

    # hf = h5py.File( os.path.join(__location__, 'data.h5'), 'r')
    # hf.keys()
    # # extract features
    # features = hf.get('features')
    # labels = hf.get('labels')
    # features = np.array(features)
    # labels = np.array(labels)
    # return features,labels

    hf = h5py.File( os.path.join(__location__, 'kbest35_mlp10000_datasetlogistic.h5'), 'r')
    # extract features
    X_train = hf.get('trainx')
    y_train = hf.get('trainy')
    X_test = hf.get('testx')
    y_test = hf.get('testy')
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    return X_train,y_train,X_test,y_test

#  Prediction Function

In [ ]:
def predictFunction(modelobj,selectorKb,new_img,spim):
  
  print(new_img.shape)
  xtest1=selectorKb.transform(new_img)
  print(xtest1.shape)
  prediction=modelobj.predict(xtest1)
  prediction=prediction.astype(int)
  print("prediction shpare",prediction.shape)
  unique_labels_testingimage=np.unique(prediction)
  label_img = np.reshape(prediction, (spim.shape[0], spim.shape[1]))
  label_img.shape
  return label_img,prediction

Visualization Function

In [ ]:

def displayClassifiedImage(spim,colorimg,new_img,label_img):
    '''
    This function takes the predicted labels and the image to be classified, 
    and adds colors to it pixel by pixel. 
    ''' 
    image=spim[:,:,0]
    thresholded_image = image > 0.1
   


    label_image = label(thresholded_image)
    props = regionprops(label_image)
    # red,gray,green,black,purple,blue,yellow,aqua,brown,magenta,orange
    color_dict={0:[255,0,0],1:[128,128,128],2:[0,128,0],3:[0,0,0],4:[128,0,128],5:[0,0,255],6:[255,255,0],7:[0,100,100],
                8:[165,42,42],9:[255,0,255],10:[255,165,0]}
    for p in props:
        region_coords=p.coords
        for pixelcoord in  region_coords:
            colorimg[pixelcoord[1],pixelcoord[0]]=color_dict[label_img[pixelcoord[1],pixelcoord[0]]]
        
    return colorimg     
        

# READ STIFF AND READMTIFF Functions

In [ ]:
def read_mtiff(filename):
    """
    Read a mask bitmap tiff.

    Mask bitmap tiff contains multiple pages of bitmap masks. The mask label
    is stored in tag 65001 in each page. The mask label is stored as an ASCII
    string that may contain unicode codepoints encoded as ASCII character
    sequences (see unicode-esca                                                         pe encoding in Python docs).

    :param filename:    filename of the mask tiff to read.
    :return:            Dict[label: str, mask: ndarray], where
                        label: the mask label
                        mask: the boolean bitmap associated with the label.
    """
    TIFFTAG_MASK_LABEL = 65001
    masks = dict()
    with TiffFile(filename) as tiff:
        for p in range(0, len(tiff.pages)):
            label_tag = tiff.pages[p].tags.get(TIFFTAG_MASK_LABEL)
            if label_tag is None:
                if p > 0:
                    print(f'** page {p}: no TIFF_MASK_LABEL tag. Ignored.')
                continue
            label = label_tag.value.encode('ascii').decode('unicode-escape')
            mask = tiff.asarray(key=p)
            masks[label] = mask > 0
    return masks

def read_stiff(filename: str, silent=False, rgb_only=False):
    """
    :param filename:    filename of the spectral tiff to read.
    :return:            Tuple[spim, wavelengths, rgb, metadata], where
                        spim: spectral image cube of form [height, width, bands],
                        wavelengths: the center wavelengths of the bands,
                        rgb: a color render of the spectral image [height, width, channels] or None
                        metadata: a free-form metadata string stored in the image, or an empty string
    """
    TIFFTAG_WAVELENGTHS = 65000
    TIFFTAG_METADATA = 65111
    spim = None
    wavelengths = None
    rgb = None
    metadata = None

    first_band_page = 0
    with TiffFile(filename) as tiff:
        # The RGB image is optional, the first band image maybe on the first page:
        first_band_page = 0
        if tiff.pages[first_band_page].ndim == 3:
            rgb = tiff.pages[0].asarray()
            # Ok, the first band image is on the second page
            first_band_page = first_band_page + 1

        multiple_wavelength_lists = False
        multiple_metadata_fields = False
        for band_page in range(first_band_page, len(tiff.pages)):
            # The wavelength list is supposed to be on the first band image.
            # The older write_tiff writes it on all pages, though, so make
            # a note of it.
            tag = tiff.pages[band_page].tags.get(TIFFTAG_WAVELENGTHS)
            tag_value = tag.value if tag else tuple()
            if tag_value:
                if wavelengths is None:
                    wavelengths = tag_value
                elif wavelengths == tag_value:
                    multiple_wavelength_lists = True
                elif wavelengths != tag_value:
                    # Well, the image is just broken then?
                    raise RuntimeError(f'Spectral-Tiff "{filename}" contains multiple differing wavelength lists!')

            # The metadata string, like the wavelength list, is supposed to be
            # on the first band image. The older write_tiff wrote it on all
            # pages, too. Make a note of it.
            tag = tiff.pages[band_page].tags.get(TIFFTAG_METADATA)
            tag_value = tag.value if tag else ''
            if tag_value:
                if metadata is None:
                    metadata = tag_value
                elif metadata == tag_value:
                    multiple_metadata_fields = True
                elif metadata != tag_value:
                    # Well, for some reason there are multiple metadata fields
                    # with varying content. This version of the function does
                    # not care for such fancyness.
                    raise RuntimeError(f'Spectral-Tiff "{filename}" contains multiple differing metadata fields!')

        # The metadata is stored in an ASCII string. It may contain back-slashed
        # hex sequences (unicode codepoints presented as ASCII text). Convert
        # ASCII string back to bytes and decode as unicode sequence.
        if metadata:
            metadata = metadata.encode('ascii').decode('unicode-escape')
        else:
            metadata = ''

        # Some of the early images may have errorneus metadata string.
        # Attempt to fix it:
        if len(metadata) >= 2 and metadata[0] == "'" and metadata[-1] == "'":
            while metadata[0] == "'":
                metadata = metadata[1:]
            while metadata[-1] == "'":
                metadata = metadata[:-1]
            if '\\n' in metadata:
                metadata = metadata.replace('\\n', '\n')

        # Generate a fake wavelength list, if the spectral tiff has managed to
        # lose its own wavelength list.
        if not wavelengths:
            wavelengths = range(0, len(tiff.pages) - 1 if rgb is not None else len(tiff.pages))

        if multiple_wavelength_lists and not silent:
            warnings.warn(f'Spectral-Tiff "{filename}" contains duplicated wavelength lists!')
        if multiple_metadata_fields and not silent:
            warnings.warn(f'Spectral-Tiff "{filename}" contains duplicated metadata fields!')

        if not rgb_only:
            spim = tiff.asarray(key=range(first_band_page, len(tiff.pages)))
            spim = np.transpose(spim, (1, 2, 0))
        else:
            spim = None

        # Make sure the wavelengths are in an ascending order:
        if wavelengths[0] > wavelengths[-1]:
            spim = spim[:, :, ::-1] if spim is not None else None
            wavelengths = wavelengths[::-1]

    # Convert uint16 cube back to float32 cube
    if spim is not None and spim.dtype == 'uint16':
        spim = spim.astype('float32') / (2**16 - 1)

    return spim, np.array(wavelengths), rgb, metadata


MAIN FUNCtion

In [ ]:
def main():
        # get the dataset
        X_train,y_train,X_test,y_test=loadDataset()
        # get the model 
        clf,selectorKb=loadModel()
        # # Pre process predict image
        spim, wl, colorimg, meta=read_stiff(current_image_loaded.tiff_img)  
        new_img = spim.reshape((spim.shape[0]*spim.shape[1]), spim.shape[2]) # (H*W,b)
        
        # # apply prediction 
        label_img,prediction=predictFunction(clf,selectorKb,new_img,spim)
        # Perform Classification and get Image
        colorimg=displayClassifiedImage(spim,colorimg,new_img,label_img)
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(colorimg)
        # Save Classified image
        imgfolderpath=settings.MEDIA_ROOT+"\\img\\"+"classifiedImg.png"
        plt.savefig(imgfolderpath)
        current_image_loaded.classified_img='img'+'/classifiedImg.png'
        current_image_loaded.save()

            # SAVE Threshold image 
        image=spim[:,:,0]
        thresholded_image = image > 0.1
        # fig, axs = plt.subplots(nrows=1, ncols=1)
        # axs.imshow(thresholded_image,'gray')
        # plt.show()
        imgfolderpath=settings.MEDIA_ROOT+"\\img\\"+"thresimg.png"
        plt.savefig(imgfolderpath)
        current_image_loaded.thres_img='img'+'/thresimg.png'
        current_image_loaded.save()